# Python Średniozaawansowany

#### Instrukcja instalacji VS Code <a class="anchor" id="install"></a>

 - Zainstaluj narzędzia [Python](https://www.python.org/downloads/).

 - Pobierz [VS Code](https://code.visualstudio.com/).

 - Zainstaluj rozszerzenie [Python](https://marketplace.visualstudio.com/items?itemName=ms-python.python).

 - Zainstaluj rozszerzenie [Jupyter](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter).

  - Zainstaluj rozszerzenie [Git Extension Pack](https://marketplace.visualstudio.com/items?itemName=donjayamanne.git-extension-pack).

#### Potrzebne importy

 - [matplotlib](https://matplotlib.org/) - wymagana instalacja
    * [pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html)
 - [numpy](https://numpy.org/) - wymagana instalacja
 - [pandas](https://pandas.pydata.org/) - wymagana instalacja
 - os
 - [inspect](https://docs.python.org/3/library/inspect.html) - biblioteka zawierająca [funkcjonalność przydatną przy analizowaniu struktur klas](https://docs.python.org/3/library/inspect.html#classes-and-functions).
 - [abc](https://docs.python.org/3/library/abc.html) - biblioteka do abstrakcji klasowej


In [1]:
from abc import ABC, abstractmethod
import traceback
import os
from inspect import getmro

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

## Wyjątki

### [Wyjątki](https://en.wikipedia.org/wiki/Exception_handling) są zdarzeniami, które pojawiają się podczas wykonania programu i przerywają jego normalny przebieg. Technicznie są to obiekty reprezentujące dany błąd.

In [3]:
a = 5
b = 0
print(a/b)

ZeroDivisionError: division by zero

##### Wszystkie wbudowane, nie opuszczające systemu [wyjątki](https://docs.python.org/3/library/exceptions.html) dziedziczą po klasie `Exception`. Wyjątki tworzone przez programistę również powinny dziedziczyć po tej klasie.

##### Najczęściej spotykane wyjątki to

 - `AssertionError` - niespełnienie warunku operacji [assert](https://docs.python.org/3/reference/simple_stmts.html#the-assert-statement)
 - `AttributeError` - odwołanie do nieistniejącego atrybutu
 - `IOError` - błąd operacji wejścia/wyjścia
 - `IndexError` - odwołanie do kolekcji za pomocą indeksu poza zakresem
 - `ImportError` - problem z załadowaniem modułu
 - `KeyError` - odwołanie do słownika nieistniejącym kluczem
 - `ValueError` - przekazanie do funkcji argumentu o niepoprawnej wartości
 - `ZeroDivisionError` - dzielenie przez zero

### Obsługa wyjątków

##### Wyjątki obsługujemy za pomocą słów kluczowych:
 - `try` - poprzedza obszar sprawdzany pod kątem wyrzucania wyjątków,
 - `except` - poprzedza obszar obsługi wyłapanego wyjątku,
 - `finally` - poprzedza obszar, który jest wykonywany pod koniec obsługi, niezależnie od tego, czy wyjątek został wyrzucony czy też nie
 - `raise` - wyrzuca zadany wyjątek

In [4]:
for i in [0.5, 0.3, 0]:
    result = 0
    try:
        if i == 0:
            result = 10000000
        else:
            result = 10/i
    except:
        result = 10000000
    print(i, " ", result)

0.5   20.0
0.3   33.333333333333336
0   10000000


In [5]:
def root4(x):
    return x ** 0.25

x = "Some dummy text"

try:
    print(root4(x))
except:
    print("Hey! I cannot take a root of that")

Hey! I cannot take a root of that


In [6]:
for i in [0.5, 0.3, 0.001]:
    result = 0
    try:
        result = 10/(max(0.1,i) - 0.1)
    except:
        result = 10000000
        raise ValueError("Value lower than 0.1")
    finally:
        print(i, " ", result)


0.5   25.0
0.3   50.00000000000001
0.001   10000000


ValueError: Value lower than 0.1

##### Do każdego bloku `try` możemy przypisać wiele bloków `except`.

In [7]:
l = [5, 0, 6, 2]

for i in range(5):
    try:
        a = 1/l[i]
        print(a)
        assert a >= 0.2
    except KeyError:
        print("Wrong key")
    except (ZeroDivisionError):
        print("Oopsie, whoopsie. You tried to divide by zero")
    except (IndexError, KeyError):
        print("I went too far")
    except:
        print("What did just happened here?")

0.2
Oopsie, whoopsie. You tried to divide by zero
0.16666666666666666
What did just happened here?
0.5
I went too far


##### Klauzulę `finally` zazwyczaj używamy, gdy po wykonaniu zadania musimy zwolnić pewne zasoby, niezależnie, czy procedura powiodła się czy nie. Na przykład, obsługa pliku.

In [8]:
dummy_file_name = "dummy.file"

try:
    file = open(dummy_file_name, "r")
    print(file.read())
except:
    print("File is not readable")
finally:
    file.close()
    if os.path.exists(dummy_file_name):
        os.remove(dummy_file_name)


File is not readable


NameError: name 'file' is not defined

### Manager kontekstu

##### Zamiast tego możemy użyć wyrażenia [with](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) zwanego managerem kontekstu, które obsługuje obiekty posiadające metody `__enter__` i `__exit__`

In [9]:
dummy_file_name = "dummy.file"

with open(dummy_file_name, "w") as file:
    file.write("Somedummytextjusttofillthefilewithsomecontent")

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

with open(dummy_file_name, "r") as file:
    print(file.read())

FileNotFoundError: [Errno 2] No such file or directory: 'dummy.file'

#### Zadanie

Napisz klasę `AutodestructingFile` która stworzy plik o lokalizacji zadanej w konstruktorze i pozwoli na zapisywanie i odczytywanie z owego pliku. Obiekt tej klasy, gdy użyty w ramach klauzuli `with`, ma usunąć automatycznie plik z dysku w momencie zakończenia działania, niezależnie od tego, czy klauzula zakończyła się poprawnie czy błędem. Należy zastąpić wszystkie wystąpienia `pass` odpowiednim kodem.

###### Rozwiązanie znajduje się w komórce poniżej kodu zadania.

In [10]:
class AutodestructingFile:

    def __init__(self, address) -> None:
        pass

    def write(self, text):
        pass

    def read(self):
        pass

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        pass


dummy_file_name = "dummy.file"
some_dummy_message = "somedummymessage"

assert not os.path.exists(dummy_file_name)

with AutodestructingFile(dummy_file_name) as file:
    assert os.path.exists(dummy_file_name)
    file.write(some_dummy_message)
    print(file.read())

    raise Exception(f"This code should fail here and {dummy_file_name} should not exist")
    print("This message should not be displayed")

AssertionError: 

##### Rozwiązanie

 * podpowiedź 1 - zapisz zmienną `address` w konstruktorze jako pole obiektu. Następnie stwórz pola `reader` i `writer` i przypisz do nich odpowiednio `open(address, "w")` i `open(address, "r")`.

 * podpowiedź 2 - w operacji `write` zapisz tekst do pliku odwołując się do pola `self.writer`. Analogicznie zczytaj treść pliku z pomocą `self.reader` w operacji `write`.

 * podpowiedź 3 - W operacji `__exit__` zamknij `self.writer` i `self.reader` oraz usuń plik z pomocą operacji `os.path.exists` i `os.remove`.

In [11]:
class AutodestructingFile:

    def __init__(self, address) -> None:
        self.address = address
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.reader.close()
        self.writer.close()
        if os.path.exists(self.address):
            os.remove(self.address)


dummy_file_name = "dummy.file"
some_dummy_message = "somedummymessage"

assert not os.path.exists(dummy_file_name)

with AutodestructingFile(dummy_file_name) as file:
    assert os.path.exists(dummy_file_name)
    file.write(some_dummy_message)
    print(file.read())

    raise Exception(f"This code should fail here and {dummy_file_name} should not exist")
    print("This message should not be displayed")

Exception: This code should fail here and dummy.file should not exist

Podobną funkcjonalność można osiągnąć również z pomocą dekoratora [@contextmanager](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager)

In [12]:
from contextlib import contextmanager

dummy_file_name = "dummy.file"

class ReadWriteFile:
    def __init__(self, address) -> None:
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()

@contextmanager
def autodestructingFile(address):
    file = ReadWriteFile(address)
    try:
        yield file
    finally:
        file.reader.close()
        file.writer.close()
        if os.path.exists(address):
            os.remove(address)


with autodestructingFile(dummy_file_name) as file:
    file.write("somedummytext")
    print(file.read())

    raise Exception("some dummy exception")
    print("message ignored")

Exception: some dummy exception

### [Rzucanie wyjątków](https://docs.python.org/3/tutorial/errors.html#raising-exceptions)

##### W celu rzucenia wyjątku wykorzystujemy wyrażenie `raise`.

In [13]:
raise Exception("My dummy exception")

Exception: My dummy exception

### Tworzenie własnych wyjątków

In [14]:
class CustomException1(Exception):
    pass

try:
    raise CustomException1("Some custom exception")
except:
    traceback.print_exc()

class CustomException2(Exception):
    def __init__(self) -> None:
        super().__init__("My hardcoded message")

    def sing(self):
        print("In the marry month of June")

try:
    raise CustomException2()
except CustomException2 as e:
    traceback.print_exc()
    e.sing()

In the marry month of June


Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_23248/2871786223.py", line 5, in <module>
    raise CustomException1("Some custom exception")
CustomException1: Some custom exception
Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_23248/2871786223.py", line 17, in <module>
    raise CustomException2()
CustomException2: My hardcoded message


###### <font color='red'>Uwaga, niektóre wyjątki zabijają jądro Jupyter. Poniższy przykład ma na celu wywołania jednego z takich wyjątków. Po śmierci jądra cały kontekst wywołania zostanie stracony i wszelkie zmienne i importy będą musiały zostać wprowadzone na nowo.</font>

Dla jakiego `n` poniższy kod może nigdy się nie zakończyć? Czemu? Czemu jednak się kończy? Jakim błędem się kończy? (wywołaj poza środowiskiem `.ipynb` żeby uzyskać treść błędu) Co to znaczy?

In [15]:
def fact(n):
    if n == 0:
        return 1
    else:
        return fact(n - 1) * n


 1. <font size="1" color="gray">Dowolna liczba nie będąca liczbą naturalną</font>
 2. <font size="1" color="gray">Funkcja będzie wywoływała siebie samą w nieskończoność. To tak zwana [rekurencja](https://en.wikipedia.org/wiki/Recursion).</font>
 3. <font size="1" color="gray">Python przerywa samodzielnie zbyt głęboko schodzące rekursje.</font>
 4. <font size="1" color="gray">Przekroczono dopuszczalną głębokość rekurencji. Przerywa to program zanim zostanie wyrzucony systemowy błąd `stackoverflow`.</font>



## [Programowanie obiektowe](https://en.wikipedia.org/wiki/Object-oriented_programming)

##### Podstawowe koncepcje to

 - [dziedziczenie](https://en.wikipedia.org/wiki/Inheritance_(object-oriented_programming)),
 - [kompozycja](https://en.wikipedia.org/wiki/Object_composition),
 - [abstrakcja](https://en.wikipedia.org/wiki/Abstraction_(computer_science)),
 - [hermetyzacja](https://en.wikipedia.org/wiki/Encapsulation_(computer_programming)),
 - [polimorfizm](https://en.wikipedia.org/wiki/Polymorphism_(computer_science)).

### Dziedziczenie

Jeżeli pewna klasa posiada funkcjonalność, która jest nam potrzebna, ale chcielibyśmy jeszcze dodać coś od siebie, możemy rozszerzyć klasę tworząc nową, która dziedziczy funkcjonalności klasy bazowej.

In [16]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name

    def eat(self, food):
        print(self.name, ":Nom, nom, nom. Tasty", food)

class Bird(Animal):
    def fly(self):
        print(self.name, ": I can fly")

animal = Animal("Creature")
animal.eat("other animal")

bird = Bird("Tweety")
bird.eat("grain")
bird.fly()

def feed(animal: Bird, food: object) -> None:
    animal.eat(food)
    animal.fly()

feed(bird, "grain")

Creature :Nom, nom, nom. Tasty other animal
Tweety :Nom, nom, nom. Tasty grain
Tweety : I can fly
Tweety :Nom, nom, nom. Tasty grain
Tweety : I can fly


Możemy sprawdzić pochodzenie naszej klasy za pomocą funkcji [getmro](https://docs.python.org/3/library/inspect.html#inspect.getmro) z pakietu `inspect`.

In [17]:
print(getmro(type(bird)))
print(getmro(type(getmro)))
print(getmro(AttributeError))
print(getmro(type(5.0)))

(<class '__main__.Bird'>, <class '__main__.Animal'>, <class 'object'>)
(<class 'function'>, <class 'object'>)
(<class 'AttributeError'>, <class 'Exception'>, <class 'BaseException'>, <class 'object'>)
(<class 'float'>, <class 'object'>)


Możemy też dziedziczyć po wielu klasach przejmując atrybuty każdej z nich.

In [18]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name
        print("I am an animal")

    def eat(self, food):
        print(self.name, ": Nom, nom, nom. Tasty", food)

class Oviparous(Animal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am an oviparous")

    def bearEgg(self):
        print(self.name, ": plop")

    def breath(self):
        print(self.name, ": wheez")

class Mammal(Animal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am a mammal")

    def giveMilk(self):
        print(self.name, ": blurp")

    def breath(self):
        print(self.name, ": phew")

class Platypus(Mammal, Oviparous):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am a platypus")

    def sting(self):
        print(self.name, ": sting")

perry = Platypus("Perry")

perry.bearEgg()
perry.giveMilk()
perry.eat("algae")
perry.breath()

print(getmro(Platypus))

I am an animal
I am an oviparous
I am a mammal
I am a platypus
Perry : plop
Perry : blurp
Perry : Nom, nom, nom. Tasty algae
Perry : phew
(<class '__main__.Platypus'>, <class '__main__.Mammal'>, <class '__main__.Oviparous'>, <class '__main__.Animal'>, <class 'object'>)


Zauważmy, że powyższa klasa rozwiązuje dziedziczenie wpierw rozpatrując klasę `Mammal`. Oznacza to, że jeśli klasy `Mammal` i `Oviparous` będą miały takie samo pole, `Platypus` przejmie tylko pole od `Mammal`.

### Kompozycja

Zamiast dziedziczyć wszystkie atrybuty możemy wkomponować jedną klasę w drugą tworząc obiekt tej pierwszej jako pole drugiej. Następnie pole to możemy wykorzystać do implementacja własnych metod.

In [19]:
from __future__ import annotations

class Hero():
    def __init__(self, name: str) -> None:
        self.name = name

    def introduce(self) -> str:
        return f"I am {self.name}"

class Mage(Hero):
    def __init__(self, name: str, staff: Staff) -> None:
        super().__init__(name)
        self.staff = staff

    def equipStaff(self, staff: Staff):
        self.staff = staff

    def castSpell(self) -> str:
        if self.staff:
            return self.staff.use()
        else:
            return None

class Staff:
    def use(self):
        pass

    def setOwner(self, mage: Mage) -> None:
        self.mage = mage

class FireWand(Staff):
    def use(self) -> str:
        return "Fireball!"

class FrostWand(Staff):
    def use(self) -> str:
        return "Brrr"

mage1 = Mage("Master", FireWand())
mage2 = Mage("Timothy", FrostWand())

print(mage1.introduce(), mage1.castSpell())
print(mage2.introduce(), mage2.castSpell())

I am Master Fireball!
I am Timothy Brrr


Zarówno Tymek jak i Mistrz potrafią czarować, potrzebują jednak do tego różdżek. Czarodziej nie musi wiedzieć, jak zostaje wykonany czar, potrzebuje tylko wiedzieć, jak korzystać z różdżki. W ten sposób rozdzielamy implementację obu klas i obiekt wkomponowany może być w dowolnym momencie zmieniony na nową implementację nie zmieniając implementacji klasy komponującej.

### Abstrakcja

Czasami chcemy stworzyć wzór klasy bez implementowania poszczególnych jej elementów. Przkładowo, chcę stworzyć klasę `Collection`, która pozwoli użytkownikowi na dostęp do elementów, ale pozwoli również zaimplementować różne kontenery przechowujące elementy.

Nie jest to podstawowy koncept w Pythonie ze względu na brak [silnej typizacji](https://en.wikipedia.org/wiki/Strong_and_weak_typing), stąd musimy wykorzystać do tego pakiet `abc`.

###### Python nie posiada silnej typizacji stąd koncept abstrakcji jest czystym formalizmem. W językach z silną typizacją zawsze musimy podać, jakiego typu dane przyjmuje funkcja. Jeżeli nie chcemy podawać gotowej implementacji od razu przy tworzeniu funkcji, możemy stworzyć klasę-wydmuszkę, która będzie tylko szablonem pod klasy obiektów przekazywanych do owej funkcji.

In [20]:
class Collection:
    def __init__(self, elements) -> None:
        pass

    def getNext(self):
        pass

c = Collection([1, 2, 4])
print(c.getNext())

class SingleElementCollection(Collection):
    def __init__(self, elements) -> None:
        super().__init__(elements)
        self.element = elements

    def getNext(self):
        return self.element

c = SingleElementCollection([1, 2, 4])

class LoopCollection(Collection):
    def __init__(self, elements) -> None:
        super().__init__(elements)
        self.elemnts = elements
        self.index = 0

    def getNext(self):
        element = self.elemnts[self.index]
        self.index = (self.index + 1) % len(self.elemnts)
        return element

c = LoopCollection([1, 2, 4])

def iterate(collection: Collection, n: int, f: function):
    for i in range(n):
        f(collection.getNext())

iterate(c, 10, print)

None
1
2
4
1
2
4
1
2
4
1


W powyższym kodzie klasa `Collection` jest w pełni poprawną klasą i można stworzyć jej obiekt. Klasa abstrakcyjna nie powinna dawać możliwości stworzenia swojej instancji. Jest to wyłącznie wydmuszka, która służy jako wzorzec do stworzenia poprawnej implementacji.

Żeby poinformować, że klasa ta jest wyłącznie wydmuszką stosujemy pakiet [abc](https://docs.python.org/3/library/abc.html). W kodzie poniżej klasa `Collection` dziedziczy po klasie `ABC` i posiada pola z dekoratorem [@abstractmethod](https://docs.python.org/3/library/abc.html#abc.abstractmethod). Z tego powodu nie można stworzyć instancji tej klasy a jedynie instancje klas dziedziczących, które implementują te pola.

In [21]:
class Collection(ABC):
    @abstractmethod
    def __init__(self, elements) -> None:
        pass

    @abstractmethod
    def getNext(self):
        pass

class SingleElement(Collection):
    def __init__(self, elements) -> None:
        self.element = elements

    def getNext(self):
        return self.element

class LoopCollection(Collection):
    def __init__(self, elements) -> None:
        self.elements = elements
        self.i = 0

    def getNext(self):
        element = self.elements[self.i]
        self.i %= len(self.elements)
        return element

c = LoopCollection([1, 2, 4])

def iterate(collection: Collection, n: int, f):
    for i in range(n):
        f(collection.getNext())

iterate(c, 10, print)

1
1
1
1
1
1
1
1
1
1


#### Zadanie

Wczytaj plik za pomocą `reader`, przetwórz treść za pomocą `converter` i zapisz do pliku za pomocą `writer`. Stwórz klasę `LUFileConverter`, która będzie konwertowała zadany plik zmieniając małe litery w wielkie i vice versa. Stwórz klasę `DoubleFileConverter`, która wykona operacje `converter` dwukrotnie.

`convertFile` w `FileConverter` ma być metodą abstrakcyjną.
`reader` ma posiadać operację `read()`, która zczyta obiekt typu `str` z pewnego pliku.
`writer` ma posiadać operację `write()`, która zapisze do pewnego pliku zadany obiekt typu `str`.
`converter` ma przyjąć obiekt typu `str` i zwrócić obiekt typu `str`.

 - Stwórz plik (`dummy.txt` lub jakikolwiek inny) z dowolną treścią (`Ala ma kota` lub cokolwiek innego).
 - Wczytaj treść tego pliku za pomocą operacji `open` i `read`.
 - Zapisz treść tego pliku za pomocą `open` i `write` do innego pliku (`temp.file` lub jakikolwiek inny).
 - Stwórz funkcję `toUpper`, która przyjmie obiekt typu `str` i zwróci ów tekst z wszystkimi małymi literami zamienionymi na wielkie.
 - Stwórz obiekt typu `FileConverter` i zapisz jako jego pola `reader`, `writer` oraz `toUpper`.
 - Zmień pole `convertFile` w `FileConverter` na metodę abstrakcyjną.

In [22]:
from abc import ABC, abstractmethod

def toUpper(text: str) -> str:
    return text.upper()

class FileConverter(ABC):
    def __init__(self, reader, writer, converter) -> None:
        self.reader = reader
        self.writer = writer
        self.converter = converter

    @abstractmethod
    def convertFile(self):
        pass

class UpperFileConverter(FileConverter):
    def __init__(self, reader, writer) -> None:
        super().__init__(reader, writer, toUpper)

    def convertFile(self):
        content = self.reader.read()
        content = self.converter(content)
        self.writer.write(content)

class DoubleFileConverter(FileConverter):
    def convertFile(self):
        content = self.reader.read()
        content = self.converter(content)
        content = self.converter(content)
        self.writer.write(content)

In [23]:
def doubleText(text):
    return text + text

in_file = "dummy.txt"
out_file = "temp.file"

with open(in_file, "w") as writer:
    writer.write("Ala ma kota\nA sierotka ma rysia\n\n")

with open(in_file, "r") as reader:
    with open(out_file, "w") as writer:
        fc = DoubleFileConverter(reader, writer, doubleText)
        fc.convertFile()

if os.path.exists(in_file):
    os.remove(in_file)
if os.path.exists(out_file):
    os.remove(out_file)

### Hermetyzacja

Niektóre pola nie powinny być dostępne z zewnątrz. Jeżeli nasza klasa wykonuje jakąś operację na własnych parametrach możemy chcieć chronić te parametry przed zmianą z zewnątrz lub przez klasy dziedziczące.

Pola [prywatne](https://en.wikipedia.org/wiki/Access_modifiers) są dostępne tylko dla klasy zawierającej owe pole. Klasy dziedziczące nie mają do tego pola dostępu. Podejście takie nazywamy hermetyzacją. Python nie posiada mechanizmu hermetyzacji i pola są zawsze dostępne. Jeżeli chcemy zaznaczyć, że pole jest zamierzone jako chronione zaczynamy jego nazwę od podkreślnika (`_`), jeżeli zaś chcemy, żeby było traktowane jako prywatne zaczynamy jego nazwę od podwójnego podkreślnika (`__`).

In [24]:
class ProtectedDummy:
    def __init__(self) -> None:
        self._counter = 0

    def add(self):
        self._counter += 1

    def getCounter(self):
        return self._counter

class ProtectedChild(ProtectedDummy):
    def add(self):
        self._counter += 3

dummy = ProtectedDummy()
dummy.add()
dummy.add()
dummy.add()
dummy._counter = -3
dummy.add()
dummy.add()
dummy.add()
dummy.getCounter()

0

Pola prywatne wciąż są osiągalne z zewnątrz jako pola o nazwie `_<CLASS_NAME><FIELD_NAME>`.

In [25]:
class PrivateDummy:
    def __init__(self) -> None:
        self.__private = 0

    def add(self):
        self.__private += 1

    def counter(self):
        return self.__private

dummy = PrivateDummy()
dir(dummy)

dummy.add()
dummy.add()
dummy.add()
dummy._PrivateDummy__private = -3
dummy.add()
dummy.add()
dummy.add()
print(dummy.counter())

0


### Polimorfizm

Python nie implementuje pełnego polimorfizmu. W językach silnie typowanych możemy zdefiniować dwie funkcje o tej samej nazwie, przyjmujące jednak parametry różnego typu.

In [26]:
def f(a: int):
    print("This is an integer")

def f(b: str):
    print("This is a string")

f(1)
f("somedummytext")

This is a string
This is a string


Python nie daje takiej możliwości i zostanie wywołana wyłącznie funkcja ostatnio zdefiniowana. Nie istnieje w nim również możliwość tworzenia funkcji o tej samej nazwie ale innej liczbie parametrów.

In [27]:
def f(a):
    print("I have got a single value")

def f(a, b):
    print("I have got two values")

f(1, 2)
f(3)

I have got two values


TypeError: f() missing 1 required positional argument: 'b'

Jedyne postacie polimorfizmu w Pythonie to funkcje wywołujące te same operacje na obiektach różnych typów, tak zwany [duck typing](https://en.wikipedia.org/wiki/Duck_typing).

In [28]:
class Duck:
    def swim(self):
        print("Duck swimming")

    def fly(self):
        print("Duck flying")

class Whale:
    def swim(self):
        print("Whale swimming")

for animal in [Duck(), Whale()]:
    animal.swim()
    animal.fly()

Duck swimming
Duck flying
Whale swimming


AttributeError: 'Whale' object has no attribute 'fly'

### [Dataclass](https://docs.python.org/3/library/dataclasses.html)

Aby nie pisać za każdym razem konstruktora (`__init__`), reprezentacji tekstowej (`__repr__`) oraz funkcjonalności porównania (`__eq__`) możemy użyć dekoratora `@dataclass`. Zapisujemy pola takiej klasy pisząc ich nazwy z typami po dwukropku.

Co typowe dla Pythona, zadane typy są wyłącznie informacyjne i nic nie stoi na przeszkodzie użyca innych typów.

In [29]:
from dataclasses import dataclass

@dataclass
class Dummy:
    a: int
    b: str
    c: list

dummy = Dummy(1, 2, 3)
dummy2 = Dummy(1, 2, 3)
print(dummy)
print(dummy == dummy2)
print(dummy.a, dummy.b, dummy.c)

Dummy(a=1, b=2, c=3)
True
1 2 3


Dektorator `@dataclass` udostępnia również możliwość tworzenie innej funkcjonalności:

 * operatory porównania (`<`, `>`, `<=` i `>=`)

In [30]:
@dataclass(order=True)
class Pair:
    a: int
    b: float

Pair(1, 2.0) < Pair(2, 1.0)

True

 * [hashowanie](https://en.wikipedia.org/wiki/Hash_function) bez gwarancji niezmienności

In [31]:
@dataclass(unsafe_hash=True)
class HashableList:
    l: list

h = HashableList(2)
print(hash(h))
h.l = 3
print(hash(h))

6909455589863252355
-5029647727744300836


 * niemutowalność pól

In [32]:
@dataclass(frozen=True)
class FrozenClass:
    a: int
    b: float

f = FrozenClass(2, 4.5)
print(f)
f.a = 4

FrozenClass(a=2, b=4.5)


FrozenInstanceError: cannot assign to field 'a'

#### Zadanie

Napisz strukturę o trzech niemutowalnych polach, którą można posortować oraz przechowywać jako element zbioru.

Dlaczego `unsafe_hash` jest odradzane w użyciu? Może to prowadzić do nieprzewidzianych zachowań przy strukturach bazujących na hashu.

In [33]:
@dataclass(order=True, unsafe_hash=True)
class Three:
    a: int
    b: int
    c: int

t1 = Three(1, 2, 3)
t2 = Three(2, 5, 3)

s = {t1, t2}
print(s)
s.add(t1)
print(s)
print(t1 in s)
t1.a = 2
print(t1 in s)
s.add(t1)
print(s)

{Three(a=2, b=5, c=3), Three(a=1, b=2, c=3)}
{Three(a=2, b=5, c=3), Three(a=1, b=2, c=3)}
True
False
{Three(a=2, b=2, c=3), Three(a=2, b=5, c=3), Three(a=2, b=2, c=3)}


### Kopia płytka a głęboka

Kopię listy możemy wykonać za pomocą konstruktora `list`, który tworzy nową listę na podstawie zadanych elementów. Kopia taka nie kopiuje jednak elementów w liście a tworzy referencje do dokładnie tych samych obiektów. Tego typu kopię nazywamy kopią płytką.

In [34]:
@dataclass
class Container:
    field: int

l1 = [Container(1), Container(2), Container(3)]
l2 = list(l1)
l2[0].field = 2
print(l1)

[Container(field=2), Container(field=2), Container(field=3)]


W opozycji do płytkiej kopii, kopia głęboka kopiuje rekurencyjnie również obiekty wewnątrz. Do tej operacji wykorzystujemy [deepcopy](https://docs.python.org/3/library/copy.html#copy.deepcopy) z biblioteki [copy](https://docs.python.org/3/library/copy.html).

In [35]:
from copy import deepcopy

@dataclass
class Container:
    field: int

l1 = [Container(1), Container(2), Container(3)]
l2 = deepcopy(l1)
l2[0].field = 2
print(l1)

c1 = Container(2)
c2 = Container(c1)
c3 = deepcopy(c2)
c3.field.field = 1
print(c2, c3)

[Container(field=1), Container(field=2), Container(field=3)]
Container(field=Container(field=2)) Container(field=Container(field=1))


## Serializacja

[Serializacją](https://en.wikipedia.org/wiki/Serialization) nazywamy proces zapisu i odczytu danych do postaci możliwej do przechowywania na dysku. Rozróżniamy dwa rodzaje formatu danych:
 - [binarny](https://en.wikipedia.org/wiki/Binary_file) - dane przechowywane są w kompaktowej formie analogicznej do formatu w pamięci podręcznej. Format mało czytelny dla człowieka i trudny do odtworzenia, jeżeli nie posiadamy informacji o kodowaniu.
 - [tekstowy](https://en.wikipedia.org/wiki/Text_file) - dane zapisane są w postaci czytelnego tekstu. Format ten zajmuje o wiele więcej miejsca, jest jednak prosty do odczytania i nie wymaga żadnych informacji o formatowaniu.

Prostym przykładem serializacji jest zapis wbudowaną funkcjonalnością `write`. Operacja ta przyjmuje wyłącznie tekst, dane w ten sposób zapisane są zawsze danymi tekstowymi. Oznacza to, że dane nietekstowe są trudne w odczycie z takiego formatu.

In [36]:
data = [1, 2, 3]

dummy_file_name = "dummy.txt"

with open(dummy_file_name, "w") as writer:
    writer.write(str(data))

with open(dummy_file_name, "r") as reader:
    l = reader.read()

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

list(l)

['[', '1', ',', ' ', '2', ',', ' ', '3', ']']

Serializacja do pliku binarnego w Pythonie odbywa się z pomocą modułów [marshal](https://en.wikipedia.org/wiki/Text_file) (wykorzystywany do operowania na pseudo-skompilowanym kodzie Pythona, niezalecane do innych zastosowań), [pickle](https://docs.python.org/3/library/pickle.html#module-pickle) czy [shelve](https://docs.python.org/3/library/shelve.html#module-shelve).

###### Moduły te nie wykonują samodzielnie operacji zapisu i odczytu a wykorzystują zadane im obiekty buforowe. Pozwala to, na przykład, na czytanie i zapisywanie danych na serwer lub korzystanie z wejścia/wyjścia do konsoli użytkownika lub innych narzędzi.

In [37]:
import pickle

data = [1, 2, 3]

dummy_file_name = "dummy.txt"

with open(dummy_file_name, "wb") as writer:
    pickle.dump(data, writer)

with open(dummy_file_name, "rb") as reader:
    l = pickle.load(reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l[1]

2

Serializacja tekstowa może odbywać się na wiele sposobów. Jednym z częstszych jest wykorzystanie tak zwanej JavaScript Object Notation lub [JSON](https://en.wikipedia.org/wiki/JSON). W Pythonie wykorzystujemy do tego moduł [json](https://docs.python.org/3/library/json.html).

In [38]:
import json

dummy_file_name = "dummy.json"

data = {
    "capitals":
    {
        "Poland": "Warsaw",
        "Sri Lanka": "Sri Jayawardenepura Kotte",
        "Czechia": "Prague"
    },
    "numbers": [1, 2, 3, 4],
    "number": 15
}

with open(dummy_file_name, "w") as writer:
    json.dump(data, writer)

with open(dummy_file_name, "r") as reader:
    l = json.load(reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l["number"]


15

###### Podobnym w strukturze formatej jest [XML](https://en.wikipedia.org/wiki/XML), do którego wykorzystujemy pakiet [xml.etree.ElementTree](https://docs.python.org/3/library/xml.etree.elementtree.html).

Formaty `JSON` i `XML` zapisują dane w formacie drzewiastym jako słowniki zagnieżdżone, to jest, są słownikami zawierającymi wartości, które tekstem, listą lub słownikiem. Inną formą zapisu danych jest zapis w postaci tablicy dwuwymiarowej. Taką formę uzyskujemy na przykład formatem [csv](https://en.wikipedia.org/wiki/Comma-separated_values). Serializujemy dane w tym formacie z pomocą modułu [csv](https://docs.python.org/3/library/csv.html).

In [39]:
dummy_file_name = "pracownicy.csv"

data = [['pracownik', 'pensja'],
    ['Jan Kowalski', '2500'],
    ['Joanna Dolegiewicz', '7500'],
    ['Katarzyna Kot', '10000']]

import csv

with open(dummy_file_name, 'w', newline='') as writer:
    csv_writer = csv.writer(writer)
    csv_writer.writerows(data)

with open(dummy_file_name, "r") as reader:
    csv_reader = csv.reader(reader)
    l = list(csv_reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l

[['pracownik', 'pensja'],
 ['Jan Kowalski', '2500'],
 ['Joanna Dolegiewicz', '7500'],
 ['Katarzyna Kot', '10000']]

Wczytywanie z pomocą modułu `csv` tworzy tablicę tablic, która może być problematyczna w operowaniu. Pliki, z których wczytujemy mogą mieć również pewne niestandardowe nagłówki i stópki a także mieć niestandardowe rozdzielanie danych. Dla lepszego operowania na większych zbiorach danych zaleca się zewnętrzny moduł [pandas](https://pandas.pydata.org/).

In [40]:
import pandas as pd

data = pd.read_csv("./samples/SBE39plus_03907421_2022_10_26.asc", sep=",", skiprows=11, header=None)
display(data)
display(data[(data[0]>1)&(data[0]<15)])
data[4] = data[1] ** 2.0 + 12
display(data)
display(data.sort_values(by=[0]))


,0,1,2,3
0,22.5229,-0.1666,26 Oct 2022,09:39:09
1,22.5217,-0.1708,26 Oct 2022,09:39:10
2,22.5207,-0.1685,26 Oct 2022,09:39:11
3,22.5196,-0.1688,26 Oct 2022,09:39:12
4,22.5195,-0.1714,26 Oct 2022,09:39:13
...,...,...,...,...
9464,13.3829,-0.1645,26 Oct 2022,12:16:53
9465,13.3952,-0.1623,26 Oct 2022,12:16:54
9466,13.4723,-0.1646,26 Oct 2022,12:16:55
9467,13.5213,-0.1628,26 Oct 2022,12:16:56


,0,1,2,3
9438,14.9072,-0.1662,26 Oct 2022,12:16:27
9439,14.8085,-0.1628,26 Oct 2022,12:16:28
9440,14.7843,-0.1673,26 Oct 2022,12:16:29
9441,14.7475,-0.1676,26 Oct 2022,12:16:30
9442,14.6152,-0.1656,26 Oct 2022,12:16:31
9443,14.4687,-0.1670,26 Oct 2022,12:16:32
9444,14.4415,-0.1641,26 Oct 2022,12:16:33
9445,14.2935,-0.1686,26 Oct 2022,12:16:34
9446,14.2080,-0.1686,26 Oct 2022,12:16:35
9447,14.0593,-0.1676,26 Oct 2022,12:16:36


,0,1,2,3,4
0,22.5229,-0.1666,26 Oct 2022,09:39:09,12.027756
1,22.5217,-0.1708,26 Oct 2022,09:39:10,12.029173
2,22.5207,-0.1685,26 Oct 2022,09:39:11,12.028392
3,22.5196,-0.1688,26 Oct 2022,09:39:12,12.028493
4,22.5195,-0.1714,26 Oct 2022,09:39:13,12.029378
...,...,...,...,...,...
9464,13.3829,-0.1645,26 Oct 2022,12:16:53,12.027060
9465,13.3952,-0.1623,26 Oct 2022,12:16:54,12.026341
9466,13.4723,-0.1646,26 Oct 2022,12:16:55,12.027093
9467,13.5213,-0.1628,26 Oct 2022,12:16:56,12.026504


,0,1,2,3,4
9462,13.2922,-0.1617,26 Oct 2022,12:16:51,12.026147
9463,13.2964,-0.1638,26 Oct 2022,12:16:52,12.026830
9461,13.3112,-0.1642,26 Oct 2022,12:16:50,12.026962
9460,13.3469,-0.1660,26 Oct 2022,12:16:49,12.027556
9464,13.3829,-0.1645,26 Oct 2022,12:16:53,12.027060
...,...,...,...,...,...
25,22.7309,-0.1688,26 Oct 2022,09:39:34,12.028493
24,22.7336,-0.1675,26 Oct 2022,09:39:33,12.028056
23,22.7371,-0.1700,26 Oct 2022,09:39:32,12.028900
22,22.7389,-0.1744,26 Oct 2022,09:39:31,12.030415


## Dekoratory

Dekorator w Pythonie to nakładka (formalnie [funkcjonał](https://en.wikipedia.org/wiki/Functional_(mathematics))), którą możemy nałożyć na funkcję. Pozwala na stworzenie kontekstu wywołania dla definiowanej funkcji. Dekorator w ten sposób jest funkcją, która zwraca funkcję, która ma zostać wykonana zamiast funkcji dekorowanej.

In [41]:
def log_usage(func):
    def wrapper():
        print("Function execution called")
        result = func()
        print(f"{result} obtained as the result of the call")
        return result
    return wrapper

@log_usage
def dummyFunction():
    return 0

dummyFunction()

Function execution called
0 obtained as the result of the call


0

Dekorator może również przyjmować argumenty, które zdefiniują jego zachowanie. W tym wypadku musimy otoczyć definicję dekoratora nakładką, która zwróci sam dekorator na podstawie zadanych argumentów.

In [42]:
def log_usage(name, greeting):
    def dec(func):
        def wrapper():
            print(f"{name}: {greeting}")
            print(f"{name}: Function execution called")
            result = func()
            print(f"{name}: {result} obtained as the result of the call")
            return result
        return wrapper
    return dec

@log_usage("Dummy logger", "Dummy hello")
def dummyFunction():
    return 0

dummyFunction()

Dummy logger: Dummy hello
Dummy logger: Function execution called
Dummy logger: 0 obtained as the result of the call


0

## Wyrażenia lambda

## Generatory i iteratory

## Wyrażenia regularne

## Wielowątkowość

## Profilowanie kodu